In [83]:
import pandas as pd
import numpy as np
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import matplotlib.pyplot as plt
import seaborn as sns

### Google Sheets Set-up 

In [84]:
# Set Up Authentication for google sheets
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("credentials.json", scope)
client = gspread.authorize(creds)

# Open Google Sheet by URL
sheet_url = "https://docs.google.com/spreadsheets/d/17kuUgLq9pMk_KvxmGSau8qpLwPOhwNH4lPg8_WLuM0s/edit?usp=sharing"
sheet = client.open_by_url(sheet_url).sheet1  # First sheet

# Convert Sheet Data to Pandas DataFrame
data = sheet.get_all_values()
df = pd.DataFrame(data)

In [85]:
# Set First Row as Column Names
df.columns = df.iloc[0]  # Assign first row as headers
df = df[1:]  # Remove the first row from data

# Drop Unnecessary Columns
df = df.drop(df.columns[9:], axis=1)  # Drop all columns from index 9 onward
print("Column names:", df.columns.tolist())  # Check the column names

df.head()

Column names: ['Timestamp', 'Temperature (C)', 'Temperature (F)', 'Humidity', 'ADC', 'Voltage', 'Pressure', 'Airspeed', 'Co2']


,Timestamp,Temperature (C),Temperature (F),Humidity,ADC,Voltage,Pressure,Airspeed,Co2
1,2025-03-11 08:36:38,29.9,85.82,64.2,N/A,N/A,N/A,N/A,N/A
2,2025-03-11 08:36:51,29.8,85.64,65.1,N/A,N/A,N/A,N/A,N/A
3,2025-03-11 08:37:02,29.8,85.64,65,N/A,N/A,N/A,N/A,N/A
4,2025-03-11 08:37:14,29.8,85.64,65,N/A,N/A,N/A,N/A,N/A
5,2025-03-11 08:37:26,29.8,85.64,65,N/A,N/A,N/A,N/A,N/A


### Data Cleaning

In [86]:
# Converting N/A values to 0 and changing data types
df['Timestamp'] = pd.to_datetime(df['Timestamp']) # Convert Timestamp to datetime
df['Temperature (C)'] = df['Temperature (C)'].replace('N/A', '0', regex=True).astype(float)
df['Temperature (F)'] = df['Temperature (F)'].replace('N/A', '0', regex=True).astype(float)
df['Humidity'] = df['Humidity'].replace('N/A', '0', regex=True).astype(float)
df['ADC'] = df['ADC'].replace('N/A', '0', regex=True).astype(float)
df['Voltage'] = df['Voltage'].replace('N/A', '0', regex=True).astype(float)
df['Pressure'] = df['Pressure'].replace('N/A', '0', regex=True).astype(float)
df['Airspeed'] = df['Airspeed'].replace('N/A', '0', regex=True).astype(float)
df['Co2'] = df['Co2'].replace('N/A', '0', regex=True).astype(float)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 920 entries, 1 to 920
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Timestamp        920 non-null    datetime64[ns]
 1   Temperature (C)  920 non-null    float64       
 2   Temperature (F)  920 non-null    float64       
 3   Humidity         920 non-null    float64       
 4   ADC              920 non-null    float64       
 5   Voltage          920 non-null    float64       
 6   Pressure         920 non-null    float64       
 7   Airspeed         920 non-null    float64       
 8   Co2              920 non-null    float64       
dtypes: datetime64[ns](1), float64(8)
memory usage: 64.8 KB


In [87]:
# Creating a column to differentiate between the timings of the data
df["Time_Diff"] = df["Timestamp"].diff().dt.total_seconds() # Calculate time difference between rows
df["Batch"] = (df["Time_Diff"] > 120).cumsum() # Create batch number based on time difference
df = df.drop(columns=["Time_Diff"])         # Drop the time difference column
latest_batch = df["Batch"].max()          # Get the latest batch number
data = df[df["Batch"] == latest_batch].copy() # Filter the latest batch data
data.head()

,Timestamp,Temperature (C),Temperature (F),Humidity,ADC,Voltage,Pressure,Airspeed,Co2,Batch
361,2025-03-16 16:41:19,27.9,82.22,62.5,0.0,0.0,0.0,0.0,725.0342,29
362,2025-03-16 16:42:07,28.1,82.58,63.4,0.0,0.0,0.0,0.0,505.0502,29
363,2025-03-16 16:42:41,28.1,82.58,63.4,0.0,0.0,0.0,0.0,699.9499,29
364,2025-03-16 16:43:14,28.1,82.58,63.5,0.0,0.0,0.0,0.0,695.0147,29
365,2025-03-16 16:43:47,28.1,82.58,64.0,0.0,0.0,0.0,0.0,715.0396,29


In [88]:
# Identify rows to drop based on the condition in CO2 column
drop_indices = []

for i in range(1, len(data) - 1):  # Start from index 1 to avoid out-of-bounds errors
    if (abs(data["Co2"].iloc[i-1] - data["Co2"].iloc[i]) > 100 and 
        abs(data["Co2"].iloc[i] - data["Co2"].iloc[i+1]) > 100 and 
        abs(data["Co2"].iloc[i-1] - data["Co2"].iloc[i+1]) < 100 and data["Co2"].iloc[i] != 0):
        
        drop_indices.append(data.index[i])  # Store index to drop

# Drop the identified rows which
data = data.drop(index=drop_indices).reset_index(drop=True)
data.head()


,Timestamp,Temperature (C),Temperature (F),Humidity,ADC,Voltage,Pressure,Airspeed,Co2,Batch
0,2025-03-16 16:41:19,27.9,82.22,62.5,0.0,0.0,0.0,0.0,725.0342,29
1,2025-03-16 16:42:41,28.1,82.58,63.4,0.0,0.0,0.0,0.0,699.9499,29
2,2025-03-16 16:43:14,28.1,82.58,63.5,0.0,0.0,0.0,0.0,695.0147,29
3,2025-03-16 16:43:47,28.1,82.58,64.0,0.0,0.0,0.0,0.0,715.0396,29
4,2025-03-16 16:44:20,28.1,82.58,64.4,0.0,0.0,0.0,0.0,735.0118,29


In [89]:
# Swap 0 with avg of previous and next values
variables = ["Co2", "Temperature (C)", "Temperature (F)", "Humidity", "ADC", "Voltage", "Pressure", "Airspeed"]

# Iterate over each variable
for v in variables:
    for i in range(1, len(data) - 1):  # Avoid index out-of-bounds errors
        if data[v].iloc[i] == 0 and data[v].iloc[i-1] != 0 and data[v].iloc[i+1] != 0:
            # Use .at[] to avoid SettingWithCopyWarning and properly update values
            data.at[data.index[i], v] = (data[v].iloc[i-1] + data[v].iloc[i+1]) / 2

data.head()


,Timestamp,Temperature (C),Temperature (F),Humidity,ADC,Voltage,Pressure,Airspeed,Co2,Batch
0,2025-03-16 16:41:19,27.9,82.22,62.5,0.0,0.0,0.0,0.0,725.0342,29
1,2025-03-16 16:42:41,28.1,82.58,63.4,0.0,0.0,0.0,0.0,699.9499,29
2,2025-03-16 16:43:14,28.1,82.58,63.5,0.0,0.0,0.0,0.0,695.0147,29
3,2025-03-16 16:43:47,28.1,82.58,64.0,0.0,0.0,0.0,0.0,715.0396,29
4,2025-03-16 16:44:20,28.1,82.58,64.4,0.0,0.0,0.0,0.0,735.0118,29


In [92]:
# Drop Batch Columns
df = df.drop(df.columns[9:], axis=1)  # Drop all columns from index 9 onward
print("Column names:", df.columns.tolist())  # Check the column names

Column names: ['Timestamp', 'Temperature (C)', 'Temperature (F)', 'Humidity', 'ADC', 'Voltage', 'Pressure', 'Airspeed', 'Co2']


### Data Visualization 

### Models 

In [90]:
def calculate_risk(co2, occupancy, airflow, exposure_time=60, room_volume=100):
    # Define the infectious dose and other parameters
    I = 0.000001  # Example value for infectious dose
    C = airflow  # Airflow rate in m³/min
    V = room_volume  # Volume of the room in m³
    t = exposure_time  # Exposure time in minutes
    P = 1 - np.exp(-I * t / (V * C))  # Infection probability formula
    return P